In [1]:
import numpy as np # giving an alas

from model import calculate_fitness # to check fitness inside reverse learning

In [2]:
# these 2 varibles givin in the document already
a_chaos = 0.5
b_chaos = 2.2

In [3]:
# these the boundrys given
LOWER_BOUND = np.array([113.8, 22.45]) 
UPPER_BOUND = np.array([114.4, 22.85])

In [4]:
def circle_chaotic_map(dim, n_pop):
	
	z = np.random.rand(n_pop, dim)

	term1 = b_chaos / (2 * np.pi)
	term2 = np.sin(2 * np.pi * z) # using eq11

	new_z = z + a_chaos - (term1 * term2) # than assamble in 1 varible 
	chaotic_pop = new_z % 1 # to kkep it between 0 and 1
    
	# one single whale in our code represents 14 different stations all at once it makes an array of 28, but we have 2, x and y cordinates, we will stretch it 14 times to reach same width as whale array 	 
	lb_tiled = np.tile(LOWER_BOUND, int(dim/2))
	ub_tiled = np.tile(UPPER_BOUND, int(dim/2))
    
	scaled_pop = lb_tiled + chaotic_pop * (ub_tiled - lb_tiled) # scaling part 
    
	return scaled_pop

In [5]:
def get_tent_random(size):

    z = np.random.rand(*size) # we should first randomize to get a number
    
    tent_values = np.where(z < 0.5, 2 * z, 2 * (1 - z)) # apply eq12
    
    return tent_values

In [6]:
def apply_reverse_learning(population, fitness_scores, demand_points):

	n_pop = population.shape[0]
	dim = population.shape[1]

	# one single whale in our code represents 14 different stations all at once it makes an array of 28, but we have 2, x and y cordinates, we will stretch it 14 times to reach same width as whale array
	lb_tiled = np.tile(LOWER_BOUND, int(dim/2))
	ub_tiled = np.tile(UPPER_BOUND, int(dim/2))

	# paper say we have to sort the population acoording to the fitness value from best lower cost to worst higher cost
	sorted_indices = np.argsort(fitness_scores)
	sorted_pop = population[sorted_indices]
	sorted_fitness = fitness_scores[sorted_indices]

	# ratio is 1 3 6
	limit_layer1 = int(n_pop * 0.1) # cut top 10
	limit_layer2 = int(n_pop * 0.4) # cut top 40 (30 + 10) 10 coming from layer 1 
    
	# giving them to new varibles
	new_pop = sorted_pop.copy()
	new_fitness = sorted_fitness.copy()

	for i in range(n_pop): # this is the top part we do nothing to them they are ok
		if i < limit_layer1:
			continue

		original_whale = sorted_pop[i]
		reverse_whale = (lb_tiled + ub_tiled) - original_whale # we are calculating opposite side of the map by eq13
        
		reverse_whale = np.clip(reverse_whale, lb_tiled, ub_tiled) # this will ensure we are not outside of the map
        
		stations_reshaped = reverse_whale.reshape(14, 2) # ok we got new position reshaped
		rev_fit = calculate_fitness(stations_reshaped, demand_points) # but is it any better
        
		if limit_layer1 <= i < limit_layer2: # for middle layer we compare it old fitness with new calculated one, if new one better we replace it, eq14 says so
			if rev_fit < sorted_fitness[i]:
				new_pop[i] = reverse_whale
				new_fitness[i] = rev_fit
                
		else: # aha bottom layer, this time we will caculate opposite but do not check if its better because its already bad 
			new_pop[i] = reverse_whale
			new_fitness[i] = rev_fit
            
		return new_pop, new_fitness

In [7]:
def iwoa_solve(demand_points, max_iter=500, pop_size=30, dim=28): # we will take 130 demand ppoint, set for 100 for now, use 30 whales, 29 dimention

	print(f"Starting IWOA Optimization for {max_iter} iterations")

	# Initialize Population (Section 2.2.1)
	population = circle_chaotic_map(dim, pop_size) # lets call the func we wrote to get random demnd points

	# Define Bounds (Shenzhen Coordinates)
	LOWER_BOUND = np.array([113.8, 22.45])
	UPPER_BOUND = np.array([114.4, 22.85])
	lb_tiled = np.tile(LOWER_BOUND, int(dim/2))
	ub_tiled = np.tile(UPPER_BOUND, int(dim/2))

	# We run a loop over every single whale. We reshape them into (14, 2) (14 stations) and ask model.py: "How much money does this configuration cost?" We save all 30 scores in the fitness array
	fitness = np.array([calculate_fitness(ind.reshape(14,2), demand_points) for ind in population])

	# Find the initial Best Whale (The Leader)
	best_idx = np.argmin(fitness) # find index of lowest cost
	best_whale = population[best_idx].copy() 
	best_score = fitness[best_idx]

	convergence_curve = [] # To track progress

	for t in range(max_iter): 
        
		population, fitness = apply_reverse_learning(population, fitness, demand_points) # we apply reverse learning if opposite better we swap it 
        
		current_best_idx = np.argmin(fitness)
		if fitness[current_best_idx] < best_score: # after swap maybe new whale is better than leader if it is than update it 
			best_score = fitness[current_best_idx]
			best_whale = population[current_best_idx].copy()
		""" # this is where WOA and IWOA differs, this commend lines for WOA
		a = 2 - 2 * t / max_iter # standart WOA update for a, it starts at 2 and linearly drop	to 0 
        
		for i in range(pop_size):

            # calculate a nd c
			r1 = np.random.rand()
			r2 = np.random.rand()
            
			A = 2 * a * r1 - a  # eq3
			C = 2 * r2          # eq4
            
			p = np.random.rand() # Strategy chooser
            
			if p < 0.5:
				if np.abs(A) < 1: # than we are close, we move towards the leader 
                    # Spiraling / Bubble Net (eq1)
					D = np.abs(C * best_whale - population[i])
					population[i] = best_whale - A * D

				else: # than we far far away 
                    # Global Search (eq9)
					rand_idx = np.random.randint(0, pop_size)
					rand_whale = population[rand_idx]
					D = np.abs(C * rand_whale - population[i])
					population[i] = rand_whale - A * D

			else: # than we ignore a, we search spiral way arount the leader 
                # Spiral Attack (eq6)
				distance_to_best = np.abs(best_whale - population[i])
				l = np.random.uniform(-1, 1)
				population[i] = distance_to_best * np.exp(l) * np.cos(2 * np.pi * l) + best_whale
				""" # from here IWOA starts
		# Nonlinear Convergence Factor 'a' (eq17)
        # Uses a cosine wave to keep diversity high in early stages
        # Uses a random 'p_prime' to decide which variation to use
		p_prime = np.random.rand()
		cosine_term = 2 * np.cos(0.5 * np.pi * (t / max_iter))
        
		if p_prime > 0.5:
			a = cosine_term + np.random.rand() * 0.01 * (1 - t/max_iter)
		else:
			a = cosine_term - np.random.rand() * 0.01 * (1 - t/max_iter)
		
		# Adaptive Probability Threshold 'adp_p' (eq15)
        # Replaces the fixed 0.5 coin flip
        # Needs 'r4' from Tent Chaos
		r4 = get_tent_random((1,))[0] 
		sin_wave = 0.2 * np.sin(100 * np.pi * (1 - t / max_iter))
		adp_p = 0.2 - (sin_wave * r4)

		# Adaptive Step Size 'U' (eq18 & 19)
        # Helps balance global vs local search
        # b = 1 is the spiral constant
		k_val = np.exp(np.random.rand() - 0.75) * (1 - t / max_iter)
		U = np.exp(k_val * 1)

		# 4. Update Positions (The Movement Logic)
		for i in range(pop_size):
            
            # IMPROVEMENT: Use Tent Map for 'r' (eq12)
			r_vec = get_tent_random((2,)) # Get 2 random numbers [r1, r2]
			r1, r2 = r_vec[0], r_vec[1]
            
            # Update Coefficients
			A = 2 * a * r1 - a  # eq3
			C = 2 * r2          # eq4
            
			p = np.random.rand() # Random number for strategy selection
            
            # IMPROVEMENT: Use 'adp_p' instead of 0.5
			if p < adp_p: 
				if np.abs(A) < 1:
                    # Shrinking Encirclement (eq1)
                    # We add 'U' to the step size if Eq. 20 applies, but let's
                    # keep standard Eq. 1 for safety unless explicitly asked to change A.
					D = np.abs(C * best_whale - population[i])
					population[i] = best_whale - A * D
				else:
                    # Global Search (eq9)
					rand_idx = np.random.randint(0, pop_size)
					rand_whale = population[rand_idx]
					D = np.abs(C * rand_whale - population[i])
					population[i] = rand_whale - A * D
            
			else: # p >= adp_p
                # Spiral Attack (eq6)
                # Note: Some IWOA versions apply 'U' here too, but standard eq6 is
				distance_to_best = np.abs(best_whale - population[i])
				l = np.random.uniform(-1, 1)
				population[i] = distance_to_best * np.exp(l) * np.cos(2 * np.pi * l) + best_whale

        # 5. Check Bounds
		population = np.clip(population, lb_tiled, ub_tiled) # if math eq find somewhere outside of map we will force it back to the edge 
        
        # 6. Calculate New Fitness
		fitness = np.array([calculate_fitness(ind.reshape(14,2), demand_points) for ind in population]) # now we will calculate the cost again 
        
        # 7. Update Leader
		current_best_idx = np.argmin(fitness)
		if fitness[current_best_idx] < best_score:
			best_score = fitness[current_best_idx]
			best_whale = population[current_best_idx].copy()
        
		convergence_curve.append(best_score) # save the score be cause we will draw a graph
        
        # Print every 10 iterations so we know it's alive
		if t % 10 == 0:
			print(f"Iteration {t}: Best Cost = {best_score:,.2f}")

	return best_whale, best_score, convergence_curve

In [8]:
if __name__ == "__main__":
    # FIX: Generate Random Demand Points within correct Shenzhen bounds
    # Longitude (X): 113.8 to 114.4
    # Latitude  (Y): 22.45 to 22.85
    demand_x = np.random.uniform(113.8, 114.4, 130)
    demand_y = np.random.uniform(22.45, 22.85, 130)
    
    # Combine them into a (130, 2) array
    mock_demand = np.column_stack((demand_x, demand_y))
    
    print("Mock Demand Points generated in Shenzhen coordinates.")
    
    # Run the Solver!
    # I increased max_iter to 100 just to see if it converges better
    best_pos, best_cost, curve = iwoa_solve(mock_demand, max_iter=500, pop_size=30)
    
    print(f"\nOptimization Finished!")
    print(f"Final Best Cost: {best_cost:,.2f} CNY")

Mock Demand Points generated in Shenzhen coordinates.
Starting IWOA Optimization for 500 iterations
Iteration 0: Best Cost = 1,474,816.35
Iteration 10: Best Cost = 1,404,994.83
Iteration 20: Best Cost = 1,402,929.18
Iteration 30: Best Cost = 1,402,218.79
Iteration 40: Best Cost = 1,402,211.78
Iteration 50: Best Cost = 1,402,163.81
Iteration 60: Best Cost = 1,382,892.15
Iteration 70: Best Cost = 1,379,848.47
Iteration 80: Best Cost = 1,378,705.51
Iteration 90: Best Cost = 1,377,854.86
Iteration 100: Best Cost = 1,377,648.65
Iteration 110: Best Cost = 1,376,611.61
Iteration 120: Best Cost = 1,366,390.54
Iteration 130: Best Cost = 1,366,360.22
Iteration 140: Best Cost = 1,366,252.25
Iteration 150: Best Cost = 1,366,247.82
Iteration 160: Best Cost = 1,366,080.77
Iteration 170: Best Cost = 1,365,986.27
Iteration 180: Best Cost = 1,365,841.90
Iteration 190: Best Cost = 1,364,642.03
Iteration 200: Best Cost = 1,364,545.37
Iteration 210: Best Cost = 1,364,524.82
Iteration 220: Best Cost = 1,36

In [9]:
# This uses the variables 'best_pos' and 'mock_demand' that are already in the notebook's memory

def audit_best_solution_corrected(stations_flat, demand_points):
    print("Function started") # If you don't see this, the call is wrong
    
    # 1. Reshape
    stations = stations_flat.reshape(14, 2)
    from model import r0, nyear, cg, phi, epsilon, gamma, lam, n_char, deg_to_km

    # 2. Construction
    capital_recovery = (r0 * (1 + r0)**nyear) / ((1 + r0)**nyear - 1)
    variable_cost_per_station = (phi * n_char) + (epsilon * n_char)
    total_investment = cg + (len(stations) * variable_cost_per_station)
    
    AC_total = total_investment * capital_recovery
    AO_total = total_investment * gamma
    F_CO = AC_total + AO_total

    # 3. Travel Time
    diff = demand_points[:, np.newaxis, :] - stations[np.newaxis, :, :]
    dists_deg = np.sqrt(np.sum(diff**2, axis=2))
    min_dists = np.min(dists_deg * deg_to_km, axis=1)
    F_Time = 365 * np.sum(min_dists) * lam

    # 4. Penalty
    violation_count = 0
    for i in range(len(stations)):
        for j in range(i + 1, len(stations)):
            if (np.sqrt(np.sum((stations[i] - stations[j])**2)) * deg_to_km) < 6.0:
                violation_count += 1
    F_Limit = (violation_count / 2) * 15000

    # 5. Print
    print("CORRECTED COST BREAKDOWN")
    print(f"Construction (F_CO):   {F_CO:,.2f} CNY")
    print(f"Travel Time (F_Time):  {F_Time:,.2f} CNY")
    print(f"Penalty (F_Limit):     {F_Limit:,.2f} CNY")
    print(f"--------------------------")
    print(f"TOTAL:                 {F_CO + F_Time + F_Limit:,.2f} CNY")

print("To be sure calling the function now")
audit_best_solution_corrected(best_pos, mock_demand)

To be sure calling the function now
Function started
CORRECTED COST BREAKDOWN
Construction (F_CO):   823,557.01 CNY
Travel Time (F_Time):  517,681.03 CNY
Penalty (F_Limit):     22,500.00 CNY
--------------------------
TOTAL:                 1,363,738.04 CNY


In [10]:
import benchmark as bench
from benchmark import get_function_details

def iwoa_solve_benchmark(func_name, max_iter = 500, pop_size=30):
    # 1. Get the math function and its specific bounds (e.g., -100 to 100)
    fitness_func, lb, ub, dim = bench.get_function_details(func_name)
    
    if fitness_func is None:
        print(f"Error: {func_name} not found.")
        return 0

    # 2. Initialize Population (Randomly within the specific bounds of the function)
    # We use simple random uniform because each function has different bounds
    population = np.random.uniform(lb, ub, (pop_size, dim))
    
    # Calculate initial fitness
    fitness = np.array([fitness_func(ind) for ind in population])
    
    best_idx = np.argmin(fitness)
    best_whale = population[best_idx].copy()
    best_score = fitness[best_idx]
    
    # 3. Main Optimization Loop
    # This is a simplified version of your IWOA logic specifically for these tests
    for t in range(max_iter):
        
        # Linear descent for convergence factor 'a'
        a = 2 - 2 * t / max_iter 
        
        for i in range(pop_size):
            r1 = np.random.rand()
            r2 = np.random.rand()
            A = 2 * a * r1 - a
            C = 2 * r2
            p = np.random.rand()
            
            if p < 0.5:
                if np.abs(A) < 1: # Encircling
                    D = np.abs(C * best_whale - population[i])
                    population[i] = best_whale - A * D
                else: # Search for Prey
                    rand_idx = np.random.randint(0, pop_size)
                    rand_whale = population[rand_idx]
                    D = np.abs(C * rand_whale - population[i])
                    population[i] = rand_whale - A * D
            else: # Spiral Update
                dist = np.abs(best_whale - population[i])
                l = np.random.uniform(-1, 1)
                population[i] = dist * np.exp(l) * np.cos(2 * np.pi * l) + best_whale
        
        # 4. Check Bounds & Update Fitness
        population = np.clip(population, lb, ub)
        fitness = np.array([fitness_func(ind) for ind in population])
        
        # Update Leader
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_score:
            best_score = fitness[current_best_idx]
            best_whale = population[current_best_idx].copy()

    # Return the final best score found
    return best_score

In [11]:
import benchmark as bench
from benchmark import get_function_details

# List of all functions to test
func_list = [f"F{i}" for i in range(1, 19)]

print(f"{'Function':<10} | {'Best Score (Fitness)':<25}")
print("-" * 40)

for func_name in func_list:
    # Get details
    fitness_func, lb, ub, dim = bench.get_function_details(func_name)
    
    if fitness_func is None:
        print(f"{func_name:<10} | Not Implemented")
        continue

    # Run Solver
    # Using the specialized benchmark solver you just defined
    best_score = iwoa_solve_benchmark(func_name, max_iter=100, pop_size=30)
    
    # Print the result nicely
    # .5e means scientific notation with 5 decimal places (e.g., 1.23456e-30)
    print(f"{func_name:<10} | {best_score:.5e}")

Function   | Best Score (Fitness)     
----------------------------------------
F1         | 4.36796e-17
F2         | 1.32910e-10
F3         | 1.44288e+05
F4         | 2.81032e-02
F5         | 2.87428e+01
F6         | 3.05749e-01
F7         | 1.34651e-02
F8         | -1.25571e+04
F9         | 3.55271e-15
F10        | 5.75788e-09
F11        | 1.11022e-16
F12        | 6.43409e-15
F13        | 5.14143e-14
F14        | 9.98004e-01
F15        | 2.04682e-02
F16        | 3.97926e-01
F17        | 3.24790e+01
F18        | 7.78270e+00
